# INDIAN START UP ECOSYSTEM

DESCRIPTIPN

In this project,we analyze and visualize industry data and identify trends and opportunities in the Indian startup market. The analysis will cover funding trends, the geographic distribution of the start-ups,funding sources, and the industrial sector in which the start-ups operate. The insights gained from this project will help venture capitalists stay ahead of the curve and identify promising investment opportunities.

<b>QUESTIONS<b>

1. Which sector or industry received  the least and most funding in each year(2018-2021).
2. Which investor provided the least and highest funding in each year(2018-2021).
3. Which Company received least and more Funds in each year (2018-2021).
4. The location received the highest and least funding in the years under review.
5. Does the Locaction of the startup impact the investments.

In [390]:
%pip install pyodbc  
%pip install python-dotenv 
%pip install summarytools
%pip install scipy




[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [391]:
# importing the necessary libraries
 
import pyodbc # Database Connectivity with  Open Database Connectivity (pyodbc)
 
from dotenv import dotenv_values # dotenv library to load environment variables
# pandas for data manipulation and analysis
import pandas as pd
import numpy as np
import warnings
 
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from summarytools import dfSummary
from scipy.stats import f_oneway
# show floats to only 4 deicmal places
pd.set_option('display.float_format', lambda x: '%0.4f' % x)

In [392]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')


# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("UID")
password = environment_variables.get("PWD")


connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"


In [393]:
# Use the connect method of the pyodbc library and pass in the connection string.
# This will connect to the server and might take a few seconds to be complete. 
# Check your internet connection if it takes more time than necessary

connection = pyodbc.connect(connection_string)

In [394]:
# Load the dataset SQL query
query1 = "Select * from dbo.LP1_startup_funding2020"
query2 = "Select * from dbo.LP1_startup_funding2021"

<b>LOADING THE DATASETS<b>

In [395]:
# Read data from the  on dataset
data_2018 = pd.read_csv('dataset\startup_funding2018.csv')
data_2019 = pd.read_csv('dataset\startup_funding2019.csv')
data_2020 = pd.read_sql(query1, connection)
data_2021 = pd.read_sql(query2, connection)
 

EXPLORATORY DATA ANALYSIS-EDA

-Here we inspect the datasets in depth year by year and column by column. This is to clean it and make it ready for analysis.


In [396]:
data_2018.head()

,Company Name,Industry,Round/Series,Amount,Location,About Company
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India","TheCollegeFever is a hub for fun, fiesta and f..."
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",A startup which aggregates milk from dairy far...
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",Leading Online Loans Marketplace in India
3,PayMe India,"Financial Services, FinTech",Angel,2000000,"Noida, Uttar Pradesh, India",PayMe India is an innovative FinTech organizat...
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,"Hyderabad, Andhra Pradesh, India",Eunimart is a one stop solution for merchants ...


<h>SPECIFIC ISSUES WITH 2018 DATASET<h>


-The location column lists the city, state and country name. This would have to be stripped to only the city name.

-2018 dataset does not have a column for investors.

-Names of some of the columns has to be changed to be consistent with those of 2019,2020 and 2021.

-Create a unique column for the year this is to make subsequent analysis easier.

-Drop the column 'About Company' it has no bearing on the subsequent analysis.

-Inspect each column separately to identify its own perculiar issues.

-In the amount column a couple of entries are in Indian Rupees. These have to be converted to US Dollars using the appriopriate exchange rate for the period under review.

In [397]:
#Create a unique year column
data_2018['Year']='2018'

In [398]:
# Drop the [About Company] column

del data_2018['About Company']

In [399]:
# Harmonizing the 2018 dataset columns to match the other three datasets
data_2018.columns=['Company_Brand','Sector','Stage','Amount($)','HeadQuarter','Year']

data_2018.head()

,Company_Brand,Sector,Stage,Amount($),HeadQuarter,Year
0,TheCollegeFever,"Brand Marketing, Event Promotion, Marketing, S...",Seed,250000,"Bangalore, Karnataka, India",2018
1,Happy Cow Dairy,"Agriculture, Farming",Seed,"₹40,000,000","Mumbai, Maharashtra, India",2018
2,MyLoanCare,"Credit, Financial Services, Lending, Marketplace",Series A,"₹65,000,000","Gurgaon, Haryana, India",2018
3,PayMe India,"Financial Services, FinTech",Angel,2000000,"Noida, Uttar Pradesh, India",2018
4,Eunimart,"E-Commerce Platforms, Retail, SaaS",Seed,—,"Hyderabad, Andhra Pradesh, India",2018


CREATE UNIQUE SECTOR AND LOCATION (CITY) COLUMNS

In [400]:
# Split the 'Sector' column
df_18 = data_2018['Sector'].str.split(pat=',', n=1, expand=True)

# Create a new column 'industry1' based on the first part of the split
data_2018['industry1'] = df_18[0]

#Drop the 'Original' Sector Column
data_2018.drop('Sector', axis=1, inplace=True)

# Rename [industry1] Column as 'New' [Sector] Column

data_2018=data_2018.rename(columns={'industry1':'Sector'})


In [401]:
# Create a new location column by splitting the HeadQuarter Column

df_2018=data_2018['HeadQuarter'].str.split(pat=',', n=1, expand=True)
data_2018['location']=df_2018[0]

# Drop the 'Old' HeadQuarter Column

data_2018.drop('HeadQuarter', axis=1, inplace=True)

# Rename the 'New' Location column as HeadQuarter
data_2018=data_2018.rename(columns={'location':'HeadQuarter'})


CHECK FOR DUPLICATES IN THE 2018 DATASET

In [402]:
df_2018.duplicated().value_counts()

True     476
False     50
dtype: int64

DROP THE DUPLICATE ENTRIES WHILST MAINTAINING THE INTEGRITY OF THE ORIGINAL DATAFRAME

In [403]:
df_2018.drop_duplicates(keep='first', inplace=True)

CONFIRM THERE ARE NO DUPLICATE ENTRIES IN THE DATAFRAME df_startup

In [404]:
df_2018.duplicated().value_counts()

False    50
dtype: int64

CURRENCY CONVERSION (Rupees to US Dollars)

In [405]:
#2018 Exchange rate Rupee to a Dollar.
exchange_rate = 68.14

def rupee_to_dollar(amount):
    if isinstance(amount, str) and amount.startswith('₹'):
        amount = float(amount.replace(',', '')[1:]) * exchange_rate
        return f'${amount:.2f}'
    else:
        return amount

# remove rupee sign and comma, and convert to dollar equivalent
data_2018['Amount($)'] = data_2018['Amount($)'].apply(lambda x: rupee_to_dollar(x))

In [406]:
data_2018.head()

,Company_Brand,Stage,Amount($),Year,Sector,HeadQuarter
0,TheCollegeFever,Seed,250000,2018,Brand Marketing,Bangalore
1,Happy Cow Dairy,Seed,$2725600000.00,2018,Agriculture,Mumbai
2,MyLoanCare,Series A,$4429100000.00,2018,Credit,Gurgaon
3,PayMe India,Angel,2000000,2018,Financial Services,Noida
4,Eunimart,Seed,—,2018,E-Commerce Platforms,Hyderabad


<h>SPECIFIC ISSUES WITH 2019 DATASET<h>

- The following three columns will be dropped as they have no bearing on the analysis to be performed subsequently-[Founded],[What it does],[Founders].

- A unique year only column has be created to enable year on year analysis to be performed.

- Change the Company/Brand column name to Company_Brand.


DROP COLUMNS THAT HAVE NO BEARING ON THE OVERALL ANALYSIS

In [407]:
# Drop the [Founded, What it does, Founders ] column

columns_to_drop = ['Founded', 'What it does', 'Founders']
data_2019.drop(columns=columns_to_drop, inplace=True)


CREATE A UNIQUE YEAR COLUMN FOR 2019 DATASET

In [408]:
# Create a unique year column
data_2019['Year']='2019'

HARMONISE THE NAMES OF THE COLUMNS; Company/Brand to Company_Brand

In [409]:
# Change the Company/Brand column name to Company_Brand
 
data_2019.rename(columns={'Company/Brand': 'Company_Brand'})

data_2019.head(3)


,Company/Brand,HeadQuarter,Sector,Investor,Amount($),Stage,Year
0,Bombay Shaving,NaN,Ecommerce,Sixth Sense Ventures,"$6,300,000",NaN,2019
1,Ruangguru,Mumbai,Edtech,General Atlantic,"$150,000,000",Series C,2019
2,Eduisfun,Mumbai,Edtech,"Deepak Parekh, Amitabh Bachchan, Piyush Pandey","$28,000,000",Fresh funding,2019


<h>SPECIFIC ISSUES WITH 2020 DATASET<h>

- The following three columns will be dropped as they have no bearing on the analysis to be performed subsequently-[Founded],[What it does],[Founders],[column10]

- A unique year only column has be created to enable year on year analysis to be performed.

- A unique year only column has be created to enable year on year analysis to be performed.

- Change [Amount] to [Amount($)].


In [410]:
# Drop the [Founded, What it does, Founders ] column

columns_to_drop = ['Founded', 'What_it_does', 'Founders','column10']
data_2020.drop(columns=columns_to_drop, inplace=True)

CREATE A UNIQUE YEAR COLUMN FOR 2020 DATASET

In [411]:
# Create a unique year column

data_2020['Year']='2020'

In [412]:
data_2020.head()

,Company_Brand,HeadQuarter,Sector,Investor,Amount,Stage,Year
0,Aqgromalin,Chennai,AgriTech,Angel investors,200000.0000,None,2020
1,Krayonnz,Bangalore,EdTech,GSF Accelerator,100000.0000,Pre-seed,2020
2,PadCare Labs,Pune,Hygiene management,Venture Center,NaN,Pre-seed,2020
3,NCOME,New Delhi,Escrow,"Venture Catalysts, PointOne Capital",400000.0000,None,2020
4,Gramophone,Indore,AgriTech,"Siana Capital Management, Info Edge",340000.0000,None,2020


<h>SPECIFIC ISSUES WITH 2020 DATASET<h>

- The following three columns will be dropped as they have no bearing on the analysis to be performed subsequently-[Founded],[What_it_does],[Founders].

- A unique year only column has be created to enable year on year analysis to be performed.

- Change [Amount] to [Amount($)].

In [413]:
# Drop the [Founded, What it does, Founders ] column

columns_to_drop = ['Founded', 'What_it_does', 'Founders']
data_2021.drop(columns=columns_to_drop, inplace=True)

In [414]:
#  Create a unique year column

data_2021['Year']='2021'

In [415]:
data_2021.rename(columns={'Amount': 'Amount($)'})

data_2021.head()

,Company_Brand,HeadQuarter,Sector,Investor,Amount,Stage,Year
0,Unbox Robotics,Bangalore,AI startup,"BEENEXT, Entrepreneur First","$1,200,000",Pre-series A,2021
1,upGrad,Mumbai,EdTech,"Unilazer Ventures, IIFL Asset Management","$120,000,000",None,2021
2,Lead School,Mumbai,EdTech,"GSV Ventures, Westbridge Capital","$30,000,000",Series D,2021
3,Bizongo,Mumbai,B2B E-commerce,"CDC Group, IDG Capital","$51,000,000",Series C,2021
4,FypMoney,Gurugram,FinTech,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","$2,000,000",Seed,2021


<b>MERGE THE FOUR (4) DATASETS TO PERFORM THE FINAL CLEANING AND ANALYSIS<b>

The four datasets have been combined into a single dataFrame called df_startup. 

The individual columns in this new DataFrame will inspected and cleaned. 

Only six relevant columns are in this new DataFrame, 
they are Company/Brand,Stage,Amount,Sector,Headquarter and Investor.

In [416]:
df_startup=pd.concat([data_2018,data_2019,data_2020,data_2021],ignore_index=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [417]:
df_startup.head()

,Company_Brand,Stage,Amount($),Year,Sector,HeadQuarter,Company/Brand,Investor,Amount
0,TheCollegeFever,Seed,250000,2018,Brand Marketing,Bangalore,NaN,NaN,NaN
1,Happy Cow Dairy,Seed,$2725600000.00,2018,Agriculture,Mumbai,NaN,NaN,NaN
2,MyLoanCare,Series A,$4429100000.00,2018,Credit,Gurgaon,NaN,NaN,NaN
3,PayMe India,Angel,2000000,2018,Financial Services,Noida,NaN,NaN,NaN
4,Eunimart,Seed,—,2018,E-Commerce Platforms,Hyderabad,NaN,NaN,NaN


In [418]:
# Drop the Company/Brand and Amount columns

df_startup.drop(['Company/Brand','Amount'], axis=1, inplace=True)


In [419]:
df_startup.head()

,Company_Brand,Stage,Amount($),Year,Sector,HeadQuarter,Investor
0,TheCollegeFever,Seed,250000,2018,Brand Marketing,Bangalore,NaN
1,Happy Cow Dairy,Seed,$2725600000.00,2018,Agriculture,Mumbai,NaN
2,MyLoanCare,Series A,$4429100000.00,2018,Credit,Gurgaon,NaN
3,PayMe India,Angel,2000000,2018,Financial Services,Noida,NaN
4,Eunimart,Seed,—,2018,E-Commerce Platforms,Hyderabad,NaN


<b>WE PERFORM A DETAILED COLUMN BY COLUMN CLEANING OF THE NEW DATAFRAME df_startup<b>

<h>ISSUES IDENTIFIED WITH THE NEW DATAFRAME THAT NEEDS TO BE ADDRESSED<h>


-The new DataFrame df_startup has 33 duplicate entries.

-The Amount column is presented in an object datatype this has to be transformed into a float.

-The year column is an object data type this has to be changed to a datetime data type.

-In the Stage column some of the stages are spelt slighty differently, these need to be harmonised and made consistent.

-In the Sector column, same sectors are spelt differently for different companies.

In [420]:
# CHECK FOR DUPLICATES

df_startup.duplicated().value_counts()

False    2846
True       33
dtype: int64

In [421]:
# DROP THE DUPLICATES ENTRIES WHILST MAINTAINING THE INTEGRITY OF THE ORIGINAL DATAFRAME

df_startup.drop_duplicates(keep='first', inplace=True)

In [422]:
df_startup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2846 entries, 0 to 2878
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company_Brand  2757 non-null   object
 1   Stage          1922 non-null   object
 2   Amount($)      614 non-null    object
 3   Year           2846 non-null   object
 4   Sector         2828 non-null   object
 5   HeadQuarter    2732 non-null   object
 6   Investor       2222 non-null   object
dtypes: object(7)
memory usage: 177.9+ KB


In [423]:
# Convert the Amount column from Object data type to Float data type.

df_startup['Amount($)'] = df_startup['Amount($)'].str.replace('$', '').str.replace(',', '').replace(['—', 'Undisclosed','Undiclosed','Undiclsosed','Undislosed'], np.nan).astype(float)

In [424]:
# Convert the year from Object to its appriopriate data type
df_startup['Year']=pd.to_datetime(df_startup.Year)


In [425]:
# Convert Year from DateTime to Year

df_startup['Year'] = df_startup['Year'].dt.year

CLEAN THE HEADQUARTER COLUMN

A couple of the entires were not fit for purpose. Some of the entries represented countries and cities not in India. India was also represented as a location instead of its cities. All these problematic entries have been dropped to alighn the entries to only the cities located in India.

In [426]:
# Drop the entries that are not relevant to the analysis to come.

df_startup.drop([12,42,706,781,791,801,838,840,844,845,847,855,875,876,877,879,880,888,889,894,897,902,907,915,916,918,921,932,984,988,999,1001,1003,1005,1006,1012,1014,1015,1028,1035,1071,1072,1073,1074,1098,1702,1713,1911,1912,1927,2571,2590,2770,2843,2846], inplace=True)

In [427]:
df_startup.drop([199,854,860,1768,2422],inplace=True)

In [428]:
df_startup.replace(to_replace=['Bangalore City','Bangalore','Banglore'], value='Bengaluru', inplace=True)
df_startup.replace(to_replace=['Jaipur, Rajastan','Rajasthan'], value='Jaipur', inplace=True)
df_startup.replace(to_replace=['Dehli'], value='New Dehli', inplace=True)
df_startup.replace(to_replace=['Tamil Nadu','Tirunelveli, Tamilnadu','The Nilgiris'], value='Chennai', inplace=True)
df_startup.replace(to_replace=['Samastipur, Bihar'], value='Samastipur', inplace=True)
df_startup.replace(to_replace=['Tumkur, Karnataka'], value='Tumkur', inplace=True)
df_startup.replace(to_replace=['Faridabad, Haryana','Dhingsara, Haryana'], value='Faridabad', inplace=True)
df_startup.replace(to_replace=['West Bengal'], value='Kolkata', inplace=True)
df_startup.replace(to_replace=['Uttar pradesh','Uttar Pradesh'], value='Lucknow', inplace=True)

In [429]:
df_startup['HeadQuarter'].unique()

array(['Bengaluru', 'Mumbai', 'Gurgaon', 'Noida', 'Hyderabad', 'Kalkaji',
       'Delhi', 'Hubli', 'New Delhi', 'Chennai', 'Mohali', 'Kolkata',
       'Pune', 'Jodhpur', 'Kanpur', 'India', 'Ahmedabad', 'Azadpur',
       'Haryana', 'Cochin', 'Faridabad', 'Jaipur', 'Kota', 'Anand',
       'Belgaum', 'Thane', 'Margão', 'Indore', 'Alwar', 'Kannur',
       'Trivandrum', 'Ernakulam', 'Kormangala', 'Lucknow', 'Andheri',
       'Mylapore', 'Ghaziabad', 'Kochi', 'Powai', 'Guntur', 'Kalpakkam',
       'Bhopal', 'Coimbatore', 'Worli', 'Alleppey', 'Chandigarh',
       'Guindy', nan, 'Telangana', 'Gurugram', 'Surat', None, 'Gujarat',
       'Kerala', 'Patna', 'Samastipur', 'Tumkur', 'Rajastan', 'Ludhiana',
       'Dehradun', 'Warangal', 'Hyderebad', 'Odisha', 'Bihar', 'Goa',
       'Ahmadabad', 'Rajsamand', 'Ranchi', 'Vadodara', 'Nagpur',
       'Samsitpur', 'Silvassa', 'Thiruvananthapuram', 'Roorkee',
       'Ambernath', 'Panchkula', 'Mangalore', 'Telugana', 'Bhubaneswar',
       'Kottayam', 'Pana

In [430]:
df_startup['HeadQuarter'].value_counts()

Bengaluru             916
Mumbai                467
Gurugram              238
New Delhi             229
Chennai               109
Pune                  104
Delhi                  88
Noida                  86
Gurgaon                80
Hyderabad              76
Jaipur                 32
Ahmedabad              28
Haryana                25
Kolkata                23
Thane                  14
Indore                 14
Chandigarh             10
Gujarat                 8
Kochi                   8
Lucknow                 7
Ahmadabad               6
Jodhpur                 5
Coimbatore              5
Ghaziabad               4
Andheri                 4
Goa                     4
Faridabad               4
Cochin                  4
Kanpur                  3
Surat                   3
Bhopal                  3
Kormangala              3
Kerala                  3
Belgaum                 2
Patna                   2
Powai                   2
Trivandrum              2
Telangana               2
Mohali      

HARMONIZING THE INDUSTRY NAME IN THE SECTOR COLUMN

In [431]:
df_startup.replace(to_replace=['Fintech'], value='FinTech', inplace=True)
df_startup.replace(to_replace=['EdTech Startup','Edtech','EdtTech'], value='EdTech', inplace=True)
df_startup.replace(to_replace=['Insurance','Banking','Credit','Consumer Lending','Accounting','Finance company','Finance','Capital Markets','Venture Capital & Private Equity','Venture capital',], value='Financial Services', inplace=True)
df_startup.replace(to_replace=['E-Commerce','Ecommerce','Social e-commerce','E-marketplace'], value='E-commerce', inplace=True)
df_startup.replace(to_replace=['Automotive & Rentals','Automobiles'], value='Automotive', inplace=True)
df_startup.replace(to_replace=['Agritech','B2B Agritech'], value='AgriTech', inplace=True)
df_startup.replace(to_replace=['Food and Beverage','Beverages','Beverage'], value='Food & Beverages', inplace=True)
df_startup.replace(to_replace=['Logistics & Supply Chain'], value='Logistics', inplace=True)
df_startup.replace(to_replace=['Information Technology & Services','Tech','Internet','IT'], value='Information Technology', inplace=True)
df_startup.replace(to_replace=['Gaming startup','Computer Games'], value='Gaming', inplace=True)
df_startup.replace(to_replace=['HealthCare','Health Care','Health,Wellness & Fitness','Fitness','Hospital & Health Care','Health','Fitness startup','Health Care','Yoga & wellness','Health & Wellness','Helath care','Healthcare','Health Diagnostics','Heathcare'], value='Healthcare', inplace=True)
df_startup.replace(to_replace=['HealthTech'], value='Healthtech', inplace=True)
df_startup.replace(to_replace=['SaaS startup','SaaS platform'], value='SaaS', inplace=True)
df_startup.replace(to_replace=['FMCG'], value='Consumer Goods', inplace=True)
df_startup.replace(to_replace=['Online Media'], value='Media', inplace=True)
df_startup.replace(to_replace=['Retail startup','Consumer'], value='Retail', inplace=True)
df_startup.replace(to_replace=['Apps','Tech Company','Technology','Information Services','Tech startup'], value='Tech Startup', inplace=True)
df_startup.replace(to_replace=['AI','Artificial Intelligence','AI Company','AI startup'], value='AI Startup', inplace=True)
df_startup.replace(to_replace=['Computer softwre','Software','Computer','Software Startup','Software Startup'], value='Computer Software', inplace=True)
df_startup.replace(to_replace=['Apparel & Fashion','Fashion startup'], value='Fashion', inplace=True)
df_startup.replace(to_replace=['B2B Service','B2B marketplace','B2B Ecommerce','B2B E-commerce','B2B startup','B2B service'], value='B2B', inplace=True)
df_startup.replace(to_replace=['Food','Foodtech','Food tech'], value='FoodTech', inplace=True)
df_startup.replace(to_replace=['Internet of Things'], value='IoT', inplace=True)
df_startup.replace(to_replace=['Farming'], value='Agriculture', inplace=True)
df_startup.replace(to_replace=['Deeptech'], value='DeepTech', inplace=True)
df_startup.replace(to_replace=['Insuretech','Insurance technology'], value='InsureTech', inplace=True)
df_startup.replace(to_replace=['Rental space'], value='Rental', inplace=True)
df_startup.replace(to_replace=['Food Delivery','Delivery Service'], value='Delivery', inplace=True)
df_startup.replace(to_replace=['Marketing & Advertising','Brand Marketing','Market Research','Marketing startup'], value='Marketing', inplace=True)
df_startup.replace(to_replace=['Biotechnology'], value='BioTechnology', inplace=True)
df_startup.replace(to_replace=['Cleantech'], value='CleanTech', inplace=True)
df_startup.replace(to_replace=['Crypto'], value='Cryptocurrency', inplace=True)
df_startup.replace(to_replace=['Interior design'], value='Interior Design', inplace=True)

STAGE COLUMN- Harmonise the various stages of investment attained by the various startups.

In [432]:
df_startup.replace(to_replace=['Seed round','Seed funding','Early seed','Seed fund','Seed Investment','Seed Round','Seed+','Seed Funding','Pre series A','Pre-series A','Pre- Series A','Seed Funding','Seed Round & Series A','Pre series A1','Funding Round'], value='Seed', inplace=True)
df_startup.replace(to_replace=['Debt Financing','Post-IPO Debt','Bridge','Bridge Round'], value='Debt', inplace=True)
df_startup.replace(to_replace=['Venture - Series Unknown','NaN','None'], value='Undisclosed', inplace=True)
df_startup.replace(to_replace=['Angel Round','Pre seed round', 'Pre Seed','Pre-seed Round','Grant'], value='Angel', inplace=True)
df_startup.replace(to_replace=['Pre-Series B','Pre-series B'], value='Pre series B', inplace=True)
df_startup.replace(to_replace=['Seis A','Series A-1','Pre series A','Series A+','Pre-series A','Series I'], value='Series A', inplace=True)
df_startup.replace(to_replace=['Post series A','Series B7','Pre-series C','Series B2','Pre series B'], value='Series B', inplace=True)
df_startup.replace(to_replace=['Corporate Round','Post-IPO Equity','Secondary Market'], value='Private Equity', inplace=True)
df_startup.replace(to_replace=['Series C,D'], value='Series D', inplace=True)
df_startup.replace(to_replace=['Series F2','Series F1'], value='Series F', inplace=True)
df_startup.replace(to_replace=['Series D'], value='Series D1', inplace=True)

<b>RESEARCH QUESTIONS<b>

1. Which Sector received the highest Funds in each year.

2. Which investor provided the most funding each year.

3. Which Company received least and more Funds in each year (2018-2021)

4. What is the funding thrend in the Indian StartUp Ecosystem?

5. Which location has received the most funding over the period under review?

QUESTION 1

Which Sector received the highest Funds in each year.

In [433]:
funding_by_year_sector = df_startup.groupby(['Year','Sector'])['Amount($)'].sum()

funding_by_year_sector = funding_by_year_sector.groupby('Year').idxmax()

for year, sector in funding_by_year_sector.items():
   print(f"{year}: {sector[1]}")

2018: Financial Services
2019: EdTech
2020: AI & Data science
2021: AI Startup


VISUALISATION

In [434]:
# Create a bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Sector', y='Year', data=funding_by_year_sector.reset_index())
plt.title('Sector with Highest Funding Each Year')
plt.xlabel('Year')
plt.ylabel('Amount($)')
plt.show()


ValueError: Could not interpret input 'Sector'

<Figure size 1000x600 with 0 Axes>

QUESTION 2

Who are the top three investors?

In [ ]:
funding_by_investor = df_startup.groupby(['Investor'])['Amount($)'].sum().reset_index()
sorted_funding_by_investor = funding_by_investor.sort_values(by='Amount($)', ascending=False)
top_investors = sorted_funding_by_investor.head(3)
top_investors['Amount($)'] = top_investors['Amount($)'].apply(lambda x: '${:,.2f}'.format(x))
top_investors

,Investor,Amount($)
1007,"MyPreferred Transformation, Avendus Finance, S...","$693,000,000.00"
1406,"South Africa’s Naspers Ventures, the CPP Inves...","$540,000,000.00"
817,"Kia Motors, Hyundai","$311,000,000.00"


VISUALISATION

QUESTION 3

What is the funding thrend of the Indian StartUp Ecosystem.

In [ ]:
funding_by_year = df_startup.groupby('Year')['Amount($)'].sum().apply(lambda x: '${:,.2f}'.format(x))
df_funding_by_year = pd.DataFrame({'Year': funding_by_year.index, 'Amount($)': funding_by_year.values})
df_funding_by_year['Amount($)'] = df_funding_by_year['Amount($)'].apply(lambda x: float(x.replace(',', '').replace('$', '')))
df_funding_by_year

,Year,Amount($)
0,2018,5797635018891.0000
1,2019,3336433200.0000
2,2020,0.0000
3,2021,0.0000


VISUALISATION

QUESTION 4

Which Locaction in India has received the most investments?

In [ ]:
funds_by_location = df_startup.groupby(['HeadQuarter'])['Amount($)'].sum().reset_index(name='Total_Raised')
funds_by_location = funds_by_location.sort_values(by='Total_Raised', ascending=False).head(5)
funds_by_location['Total_Raised'] = funds_by_location['Total_Raised'].apply(lambda x: '${:,.2f}'.format(x))

funds_by_location

,HeadQuarter,Total_Raised
78,Mumbai,"$1,410,332,828,500.00"
46,Haryana,"$1,404,629,480,000.00"
81,New Delhi,"$1,033,754,386,000.00"
15,Bengaluru,"$581,611,313,846.00"
54,Jaipur,"$380,974,925,000.00"


VISUALISATION